<a href="https://colab.research.google.com/github/EllisBuxton/PPIT-Project/blob/main/MelodyGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Genetic Alogirthm Melody Generation**

In this notebook I will be creating a Genetic Algorithm and using it to create a melody.

I hope to show the following :    

1.   What are Genetic Algorithm's.
2.   How they work.
3.   Generating Melody's using the Genetic Algorithm





# Setup

The **random** module is used in this genetic algorithm implementation to introduce randomness at various stages, such as generating random genomes, selecting individuals from the population, performing mutation, and determining crossover points. Randomness is a crucial aspect of genetic algorithms because it helps explore the search space effectively, preventing the algorithm from getting stuck in local optima and promoting diversity within the population.

In [1]:
from random import choices, randint, randrange, random, sample

The **typing** module is used to provide type hints for function parameters and return values. Type hints improve code readability and maintainability by indicating the expected types of arguments and return values for functions.

In [2]:
from typing import List, Optional, Callable, Tuple, Dict

The **os** Module is used to provide access to operating system functionalities. In this script, it's used for creating directories and handling file paths

In [3]:
import os

The **time** module provides access to time-related functions. It's used for introducing delays in the script, such as waiting before playing the next generation of melodies.

In [4]:
import time

The **datetime** module is used for generating unique folder names based on the current timestamp.

In [5]:
from datetime import datetime

**MidiUtil** and **Pyo** may need to be installed using pip.

In [6]:
!pip install pyo
!pip install midiutil


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\ellis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\ellis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


The **midiutil** module is used for generating MIDI files based on the melodies generated by the genetic algorithm.

In [7]:
from midiutil import MIDIFile

The **pyo** module is a Python library for digital signal processing, synthesis, and audio programming. It's used in this script for sound synthesis, generating metronome sounds, and playing melodies.

In [8]:
import pyo
from pyo import *


WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter (if available).
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



This notebook will be split into 2 parts, I will go into detail on the Genetic Algorithm itself and then explain how i used it to create melody's

# Introduction:

A genetic algorithm (GA) is a type of optimization algorithm inspired by the principles of natural selection and genetics. It is commonly used to find optimal or near-optimal solutions to complex problems where traditional methods might be impractical.

The basic idea behind genetic algorithms is to mimic the process of natural selection to evolve solutions to a problem over successive generations.



# How it works:

1. Initialization: A population of potential solutions (individuals/Genome's) is randomly generated.

2. Evaluation: Each individual in the population is evaluated according to a predefined fitness function, which measures how good the solution is. This function is problem-specific and typically reflects the objective or goal of the optimization.

3. Selection: Individuals are selected from the current population to serve as parents for the next generation. Selection is usually based on the individuals' fitness, with fitter individuals being more likely to be selected. Various selection methods such as roulette wheel selection, tournament selection, or rank-based selection can be used.

4. Crossover (Recombination): Selected individuals (parents) are combined to produce offspring through crossover or recombination. This process involves exchanging genetic information between parents to create new solutions. The specific method of crossover varies but often involves randomly selecting crossover points in the genetic representation of individuals.

5. Mutation: After crossover, some individuals may undergo mutation, which introduces small random changes to their genetic representation. Mutation helps to maintain genetic diversity in the population and prevents premature convergence to suboptimal solutions.

6. Replacement: The new offspring and possibly some of the existing individuals form the next generation population, replacing the previous generation.

7. Termination: The algorithm continues iterating through generations until a termination criterion is met, such as reaching a maximum number of generations, finding a satisfactory solution, or no significant improvement occurring over several generations.

# The Algorithm:

* Starting off im defining type aliases using Python annotations. Type aliases
allow you to assign a new name to an existing type. This can enhance code readability, maintainability, and flexibility, especially in situations where complex types are frequently used.

In [9]:
# Type Aliases
Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

* Here I'm defining a function called ***generate_genome*** that takes an integer parameter length, representing the length of the genome to generate. The function returns a random genome, which is essentially a sequence of binary digits (0s and 1s) of the specified length.

In [10]:
# Function to generate a random genome of specified length
def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)

* the ***generate_population*** takes in an integer paramter size, representing the size of the population to generate. It also takes in the length of the genomes from the ***generate_genome*** function. It returns the desired amount of genome's at the desired length.

In [11]:
# Function to generate a population of genomes
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

* This function is implementing a ***single-point crossover*** operation between two genomes a and b. In genetic algorithms and evolutionary computation, crossover is a fundamental genetic operator used to create offspring from parent genomes. It simulates the natural genetic recombination process that occurs during sexual reproduction.

    Here's what is happening :




1. **Input Validation**: The function starts by ensuring that both genomes a and b have the same length. This is crucial for the crossover operation to be performed properly. If the lengths are not equal, it raises a ValueError.

2. **Length Check**: If the length of the genomes is less than 2, it means there's not enough genetic material to perform a meaningful crossover. In this case, it simply returns the original genomes a and b. This is to prevent errors and ensure that the crossover operation can be meaningful.

3. **Selecting Crossover Point**: A random integer p is chosen between 1 and length - 1, inclusive. This integer represents the index at which the crossover will occur. It's important that p is not 0 or equal to the length of the genomes, as that wouldn't result in any crossover.

4. **Performing Crossover**: The crossover operation is then performed. The first part of genome a up to index p is combined with the second part of genome b starting from index p. Similarly, the first part of genome b up to index p is combined with the second part of genome a starting from index p. This swapping of genetic material simulates the exchange of genetic information between two parent genomes.

5. **Returning Offspring Genomes**: The function returns a tuple containing the two offspring genomes resulting from the crossover operation.

In [12]:
# Function to perform crossover between two genomes
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of the same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

* This function is implementing a mutation operation on a genome. In evolutionary algorithms, mutation introduces random changes in individual genomes to explore new regions of the search space and potentially improve the diversity of the population. Here's an explanation of what's happening in the function:

Input Parameters:
* **genome**: The genome on which mutation will be performed.
* **num**: The number of mutations to perform. By default, it's set to 1, meaning the function will mutate one randomly selected gene in the genome.
* **probability**: The probability of each gene in the genome to undergo mutation. By default, it's set to 0.5, meaning each gene has a 50% chance of being mutated.

Mutation Process:
* The function iterates num times, each time selecting a random index within the genome.
* At each iteration, a random index within the genome is selected using randrange(len(genome)). This index determines which gene will undergo mutation.
* The gene at the selected index is then mutated based on the probability parameter. If a random number generated by random() is greater than probability, the gene remains unchanged (genome[index] = genome[index]). Otherwise, the gene is flipped (if it's binary) by subtracting its value from 1 (abs(genome[index] - 1)).

Returning Mutated Genome:
* Once all mutations are performed, the function returns the mutated genome.


In [13]:
# Function to perform mutation on a genome
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = random.randrange(len(genome))
        genome[index] = genome[index] if random.random() > probability else abs(genome[index] - 1)
    return genome

* This function calculates the total fitness of a population by summing up the fitness values of all the genomes within that population. Here's a breakdown of what's happening in the function:

Input Parameters:

* population: This is a collection of genomes representing the entire population.
* fitness_func: This is a fitness function that takes a single genome as input and returns its fitness value. This function is applied to each genome in the population to calculate its fitness.

Comprehension:

* The function uses a list comprehension to iterate over each genome in the population.
For each genome, the fitness_func is called to calculate its fitness value. This results in a list of fitness values corresponding to each genome in the population.
Summation:

* The sum() function is then used to sum up all the fitness values obtained from the previous step.
This sum represents the total fitness of the entire population.
Return:

* Finally, the total fitness value of the population is returned.
This function essentially encapsulates the process of evaluating the fitness of each genome in the population and aggregating these fitness values to provide an overall assessment of the population's fitness. It's a fundamental step in many evolutionary algorithms where the fitness of individuals determines their likelihood of being selected for reproduction and further evolution.

In [14]:
# Function to calculate the total fitness of a population
def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum(fitness_func(genome) for genome in population)

This function selects a pair of genomes from the population for reproduction. It utilizes a method called roulette wheel selection, where genomes are selected with a probability proportional to their fitness. Here's what's happening in the function:

Input Parameters:

* population: This is a collection of genomes representing the entire population.
fitness_func: This is a fitness function that takes a single genome as input and returns its fitness value.

Generating Weighted Distribution:

* The function generate_weighted_distribution() is called with the population and fitness function as arguments. This function generates a weighted distribution of genomes based on their fitness values. Genomes with higher fitness have a higher probability of being selected.

Sampling Pair of Genomes:

* The sample() function is then called to randomly select 2 genomes from the weighted distribution.
The k parameter is set to 2, indicating that we want to select 2 genomes.

Return:
* The function returns the selected pair of genomes.

In summary, this function implements a method of selecting genomes for reproduction where genomes with higher fitness are more likely to be selected, thus promoting the propagation of fitter individuals in the population over successive generations.

In [15]:
# Function to select a pair of genomes from the population for reproduction
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )

This function generate_weighted_distribution aims to create a weighted distribution of genomes from the population based on their fitness values. Here's a breakdown of what's happening:

Input Parameters:

* population: This is a collection of genomes representing the entire population.
*fitness_func: This is a fitness function that takes a single genome as input and returns its fitness value.

Initialization:

* An empty list result is initialized. This list will contain genomes with duplicates based on their fitness.

Generating Weighted Distribution:

* The function iterates over each genome in the population.
For each genome, its fitness value is calculated using the fitness_func.
The fitness value is then converted into an integer (by adding 1) and used to determine how many times the genome should be added to the result list. This means genomes with higher fitness will appear more times in the result list, making them more likely to be selected.

Return:
* The function returns the generated weighted distribution, where genomes are duplicated based on their fitness values.

In essence, this function creates a distribution where genomes with higher fitness have a higher probability of being selected for reproduction, thus promoting the propagation of fitter individuals in the population during selection processes such as roulette wheel selection.

In [16]:
# Function to generate a weighted distribution for selection
def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []

    for gene in population:
        result += [gene] * int(fitness_func(gene) + 1)

    return result

This function, sort_population, sorts the population based on the fitness of the genomes using a custom key. Here's a breakdown of what's happening:

Input Parameters:

* population: This is a collection of genomes representing the entire population.
* fitness_func: This is a fitness function that takes a single genome as input and returns its fitness value.

Sorting:

* The sorted() function is used to sort the population.
* The key parameter is set to fitness_func, which means the sorting will be based on the fitness value of each genome. The fitness_func is applied to each genome, and the resulting fitness values are used as the sorting criteria.
* The reverse parameter is set to True, which means the population will be sorted in descending order of fitness. This is because higher fitness values are usually considered better in evolutionary algorithms.

Return:
* The function returns the sorted population.

In summary, this function sorts the population based on the fitness values of the genomes, ensuring that the fittest individuals are positioned at the beginning of the population. This sorting step is often crucial in evolutionary algorithms to facilitate selection and reproduction mechanisms.







In [17]:
# Function to sort the population based on fitness
def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)

This is just a simple funtion that converts a genome to a string so it can be printed out

In [18]:
# Function to convert a genome to a string for printing
def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

This just prints out some statistics about the current population

In [19]:
# Function to print statistics of the current population
def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join(genome_to_string(gene) for gene in population))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]

This run_evolution function is responsible for executing a genetic algorithm. It iterates through generations of populations, performing selection, crossover, and mutation operations to evolve the population towards individuals with higher fitness values. Here's a breakdown of what's happening in the function:

Input Parameters:

* populate_func: This function is responsible for generating an initial population.
* fitness_func: This function evaluates the fitness of each genome in the population.
* fitness_limit: This is the target fitness value. If any genome reaches this fitness value, the evolution process terminates.
* selection_func: This function selects pairs of genomes from the population for reproduction. By default, it's set to selection_pair.
* crossover_func: This function performs crossover between pairs of parent genomes. By default, it's set to single_point_crossover.
* mutation_func: This function performs mutation on genomes. By default, it's set to mutation.
* generation_limit: This specifies the maximum number of generations to evolve the population. By default, it's set to 100.
* printer: This function is optional and can be used to print information about the population at each generation.

Initializing Population:
* The initial population is generated using the populate_func().
Evolution Loop:

* The function iterates through generations of the population, with a limit set by generation_limit.

At each generation:

* The population is sorted based on the fitness of the genomes.
* If a printer function is provided, it's called to print information about the population.
* If the fitness of the fittest genome in the population meets or exceeds the fitness_limit, the evolution process terminates.
Otherwise, a new generation is created:
* The fittest individuals from the current generation are carried over to the next generation (next_generation).
* Additional offspring are generated through selection, crossover, and mutation operations and added to the next generation.
The current population is updated to the newly generated population (next_generation).

Return:
* The function returns the final population and the number of generations elapsed (i).

In summary, this function orchestrates the genetic algorithm process, from initializing the population to evolving it across generations until a termination condition is met. It combines various genetic operators to drive the evolution of the population towards individuals with higher fitness values.

In [20]:
# Function to run the genetic algorithm
def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

# Melody Generation with Genetic Algorithm Implementation:

In the below code i've used the genetic algorithm to generate musical melodies through a process of evolution.

These area constants that will be used in realation to Melody/Music generation.


BITS_PER_NOTE:

* This constant represents the number of bits used to encode each note in the melody. It's used in the genome representation, where each note is encoded as a sequence of binary digits.

KEYS:

* This constant represents a list of musical keys. Each element in the list corresponds to a specific key, such as "C", "C#", "Db", etc. These keys are used in music theory to define the tonal center or starting note of a musical composition.

SCALES:

* This constant represents a list of musical scales or modes. Each element in the list corresponds to a specific scale or mode, such as "major", "minor", "dorian", etc. These scales define the sequence of intervals used to construct melodies and harmonies in a given key.

In [21]:
# Constants
BITS_PER_NOTE = 4
KEYS = ["C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#", "Ab", "A", "A#", "Bb", "B"]
SCALES = ["major", "minorM", "dorian", "phrygian", "lydian", "mixolydian", "majorBlues", "minorBlues"]

This function int_from_bits converts a list of bits (binary digits) into an integer value. Here's a breakdown of what it does:

Function Signature:

* int_from_bits(bits: List[int]) -> int: It takes a list of integers (bits) as input and returns an integer.

Explanation:

* The function iterates over each bit in the list using a list comprehension.
For each bit, it multiplies the bit value (bit) by 2 raised to the power of its index in the list (index). This is done using the pow() function.
The resulting values are summed up using the sum() function.
* The final sum represents the binary value converted into an integer.

Example:

* Suppose you have a list of bits [1, 0, 1, 1].
For the first bit (1), its value is multiplied by 2^0 (1), resulting in 1.
For the second bit (0), its value is multiplied by 2^1 (2), resulting in 0.
For the third bit (1), its value is multiplied by 2^2 (4), resulting in 4.
For the fourth bit (1), its value is multiplied by 2^3 (8), resulting in 8.
Summing these values (1 + 0 + 4 + 8) yields 13, which is the integer representation of the binary sequence [1, 0, 1, 1].

Return Value:

* The function returns the integer value obtained from the binary representation.

In [22]:
# Helper function to convert a list of bits to an integer
def int_from_bits(bits: List[int]) -> int:
    return int(sum([bit*pow(2, index) for index, bit in enumerate(bits)]))

This function, prompt_for_input, is designed to interactively prompt the user for input, with the option to provide a default value and specify the desired data type of the input.

In [23]:
# Function to prompt for user input with optional default value and type casting
def prompt_for_input(prompt: str, default=None, type_=None):
    user_input = input(prompt + (f" [{default}]" if default is not None else "") + ": ")
    if user_input == "" and default is not None:
        return default
    if type_ is not None:
        try:
            return type_(user_input)
        except ValueError:
            print("Invalid input. Please try again.")
            return prompt_for_input(prompt, default, type_)
    return user_input


This function, genome_to_melody, converts a genome (represented as a sequence of bits) into a melody representation. Here's a breakdown of what it does:

Function Signature:

* genome_to_melody:  The function iteself takes a genome, along with several parameters defining the melody, and returns a dictionary containing the melody representation.

Explanation:

Extracting Notes from Genome:
* The function divides the genome into chunks, with each chunk representing a note. It iterates over the genome, slicing it into segments of length BITS_PER_NOTE.
* Each segment represents a single note in the melody.

Defining Note Length and Scale:
* The function calculates the note length based on the total number of notes (num_notes) in a bar. The length of a note is determined by dividing the total duration of a bar (assumed to be 4) by the number of notes per bar (num_notes).
* It defines the musical scale (scl) based on the specified key, scale, and root note. This scale will be used to map the notes extracted from the genome to actual musical notes.

Converting Notes to Melody Representation:

* It iterates over the extracted notes from the genome.
* For each note, it converts the binary representation into an integer using the int_from_bits function.
* If pauses are allowed (pauses is not zero), it ensures that the integer value of the note is within a valid range by taking its modulo with pow(2, BITS_PER_NOTE - 1). This step handles the case where the binary representation exceeds the maximum allowed value.
* Based on the converted integer value, it populates the melody representation:
* If the integer value represents a pause, it adds a rest in the melody with zero velocity and a note length defined earlier.
* Otherwise, it adds the actual musical note to the melody representation along with its velocity and the corresponding note length.

Adjusting Melody Steps:

* It adjusts the melody steps by applying the defined scale to each note in the melody. This step ensures that the generated melody adheres to the specified musical scale.

Return Value:

* The function returns a dictionary containing three lists:
"notes": The musical notes of the melody, adjusted based on the defined scale.
"velocity": The velocity (loudness) of each note in the melody.
"beat": The duration of each note in the melody.

In [24]:
# Function to convert a genome to melody representation
def genome_to_melody(genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                     pauses: int, key: str, scale: str, root: int) -> Dict[str, list]:
    # Extract notes from the genome
    notes = [genome[i * BITS_PER_NOTE:i * BITS_PER_NOTE + BITS_PER_NOTE] for i in range(num_bars * num_notes)]

    note_length = 4 / float(num_notes)

    # Define the scale
    scl = EventScale(root=key, scale=scale, first=root)

    melody = {
        "notes": [],
        "velocity": [],
        "beat": []
    }

    # Convert notes to melody representation
    for note in notes:
        integer = int_from_bits(note)

        if pauses:
            integer = int(integer % pow(2, BITS_PER_NOTE - 1))

        if integer >= pow(2, BITS_PER_NOTE - 1):
            melody["notes"] += [0]
            melody["velocity"] += [0]
            melody["beat"] += [note_length]
        else:
            if len(melody["notes"]) > 0 and melody["notes"][-1] == integer:
                melody["beat"][-1] += note_length
            else:
                melody["notes"] += [integer]
                melody["velocity"] += [127]
                melody["beat"] += [note_length]

    steps = []
    for step in range(num_steps):
        steps.append([scl[(note+step*2) % len(scl)] for note in melody["notes"]])

    melody["notes"] = steps
    return melody

This function, genome_to_events, takes a genome and other parameters related to the melody, and converts it into a sequence of events suitable for sound synthesis by Pyo. Here's an explanation of what it does:

Function Signature:

* genome_to_events(genome: Genome, num_bars: int, num_notes: int, num_steps: int, pauses: bool, key: str, scale: str, root: int, bpm: int) -> [Events]: It takes a genome along with other parameters defining the melody and returns a list of events.

Explanation:

Converting Genome to Melody:

* It first calls the genome_to_melody function to convert the genome into a melody representation. This step generates a dictionary containing the musical notes, velocities, and durations of the melody based on the genome and other parameters.

Creating Events:

* It iterates over each step (bar) in the melody.
* For each step, it creates an Events object:
* midinote: Represents the MIDI note numbers corresponding to the notes in the melody. It uses an EventSeq to specify a sequence of MIDI notes for the step.
* midivel: Represents the MIDI velocity (loudness) of each note in the melody. It uses an EventSeq to specify a sequence of velocities for the step.
* beat: Represents the duration of each note in the melody. It uses an EventSeq to specify a sequence of durations for the step.
attack, decay, sustain, release: Parameters controlling the envelope of the synthesized sound. These values determine how the sound evolves over time.
* bpm: Beats per minute, specifying the tempo of the melody.
List Comprehension:
It uses a list comprehension to create an Events object for each step in the melody, resulting in a list of Events objects.
Return Value:

The function returns a list of Events objects, each representing a step in the melody with MIDI note numbers, velocities, durations, and other parameters required for sound synthesis.

In [25]:
# Function to convert a genome to events for sound synthesis
def genome_to_events(genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                     pauses: bool, key: str, scale: str, root: int, bpm: int) -> [Events]:
    melody = genome_to_melody(genome, num_bars, num_notes, num_steps, pauses, key, scale, root)

    return [
        Events(
            midinote=EventSeq(step, occurrences=1),
            midivel=EventSeq(melody["velocity"], occurrences=1),
            beat=EventSeq(melody["beat"], occurrences=1),
            attack=0.001,
            decay=0.05,
            sustain=0.5,
            release=0.005,
            bpm=bpm
        ) for step in melody["notes"]
    ]

This function, fitness, is responsible for evaluating the fitness of a given genome based on its generated melody. Here's a breakdown of what it does:

Function Signature:

* fitness(genome: Genome, s: Server, num_bars: int, num_notes: int, num_steps: int, pauses: bool, key: str, scale: str, root: int, bpm: int) -> int: It takes a genome, a server object, and various parameters related to the melody generation, and returns an integer representing the fitness rating.
Explanation:

Metronome Initialization:

* It initializes a metronome (m) with the provided beats per minute (bpm) using the metronome function.
Generating Events:
It calls the genome_to_events function to convert the genome into a sequence of events representing the melody. These events are played by the server (s).

Playback and Rating:

* It starts the server (s) to play the generated melody events.
* It prompts the user to provide a rating for the melody, expecting a value between 0 and 5.

Rating Conversion:

* It attempts to convert the user-provided rating to an integer. If the conversion fails (e.g., if the user inputs a non-integer value), it defaults the rating to 0.

Server and Delay Handling:

* It stops the server (s) to halt the playback of the melody.
It introduces a 1-second delay using time.sleep(1) to ensure proper termination and avoid potential conflicts with subsequent evaluations.
Return Value:

The function returns the user-provided rating as an integer, representing the fitness of the genome. This rating indicates how well the generated melody is perceived by the user, with higher ratings indicating better fitness.

In [26]:
# Function to evaluate the fitness of a genome
def fitness(genome: Genome, s: Server, num_bars: int, num_notes: int, num_steps: int,
            pauses: bool, key: str, scale: str, root: int, bpm: int) -> int:
    m = metronome(bpm)

    events = genome_to_events(genome, num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
    for e in events:
        e.play()
    s.start()

    rating = input("Rating (0-5)")

    for e in events:
        e.stop()
    s.stop()
    time.sleep(1)

    try:
        rating = int(rating)
    except ValueError:
        rating = 0

    return rating


This function, metronome, is responsible for generating a metronome sound.

In [27]:
# Function to generate a metronome sound
def metronome(bpm: int):
    met = Metro(time=1 / (bpm / 60.0)).play()
    t = CosTable([(0, 0), (50, 1), (200, .3), (500, 0)])
    amp = TrigEnv(met, table=t, dur=.25, mul=1)
    freq = Iter(met, choice=[660, 440, 440, 440])
    return Sine(freq=freq, mul=amp).mix(2).out()

This function creates a midi file of a genome and saves it to a folder

In [28]:
# Function to save the melody generated from a genome to MIDI file
def save_genome_to_midi(filename: str, genome: Genome, num_bars: int, num_notes: int, num_steps: int,
                        pauses: bool, key: str, scale: str, root: int, bpm: int):
    melody = genome_to_melody(genome, num_bars, num_notes, num_steps, pauses, key, scale, root)

    if len(melody["notes"][0]) != len(melody["beat"]) or len(melody["notes"][0]) != len(melody["velocity"]):
        raise ValueError

    mf = MIDIFile(1)

    track = 0
    channel = 0

    time = 0.0
    mf.addTrackName(track, time, "Sample Track")
    mf.addTempo(track, time, bpm)

    for i, vel in enumerate(melody["velocity"]):
        if vel > 0:
            for step in melody["notes"]:
                mf.addNote(track, channel, step[i], time, melody["beat"][i], vel)

        time += melody["beat"][i]

    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "wb") as f:
        mf.writeFile(f)

In this code, the main function orchestrates the entire genetic algorithm process for generating melodies.

Here's whats happening :

Prompting for Parameters:

* The function prompts the user to input various parameters such as the number of bars, notes per bar, number of steps, whether to introduce pauses, key, scale, scale root, population size, number of mutations, mutation probability, and BPM (beats per minute).

Generating Initial Population:

* The initial population of genomes is generated. Each genome represents a melody. The size of the population is determined by the population_size parameter.

Evaluation and Selection:

* The fitness of each genome in the population is evaluated using the fitness function.
* Genomes are evaluated based on how well their melodies sound when played. * *
* Higher fitness values indicate better melodies.
The population is sorted based on fitness in descending order.
The top two genomes (melodies) are selected for playback and further reproduction.

Reproduction:

* For the next generation, offspring genomes are generated through selection, crossover, and mutation processes. This is done by selecting pairs of parent genomes, applying crossover to create offspring, and mutating the offspring genomes.
* The top two genomes from the previous generation are retained, and the rest of the population is filled with offspring.
Playback and MIDI File Saving:

* The top two genomes from the current generation (population) are played back using the genome_to_events function to hear their melodies.
The melodies of all genomes in the population are saved as MIDI files using the save_genome_to_midi function.
Loop and User Interaction:

The process continues until the user decides to stop. After each generation, the user is prompted to continue or stop the algorithm.
This function essentially implements the core logic of a genetic algorithm for generating melodies, including evaluation, selection, reproduction, and user interaction.

In [ ]:
# Main function
def main():
    # Prompt user for various parameters
    num_bars = prompt_for_input("Number of bars", default=8, type_=int)
    num_notes = prompt_for_input("Notes per bar", default=4, type_=int)
    num_steps = prompt_for_input("Number of steps", default=1, type_=int)
    pauses = prompt_for_input("Introduce Pauses (True/False)", default=False, type_=bool)
    key = prompt_for_input("Key", default="C")
    scale = prompt_for_input("Scale", default="major")
    root = prompt_for_input("Scale Root", default=4, type_=int)
    population_size = prompt_for_input("Population size", default=10, type_=int)
    num_mutations = prompt_for_input("Number of mutations", default=2, type_=int)
    mutation_probability = prompt_for_input("Mutation probability", default=0.5, type_=float)
    bpm = prompt_for_input("BPM", default=128, type_=int)

    folder = str(int(datetime.now().timestamp()))

    # Generate initial population
    population = [generate_genome(num_bars * num_notes * BITS_PER_NOTE) for _ in range(population_size)]

    s = Server().boot()

    population_id = 0

    running = True
    while running:
        # Shuffle the population
        random.shuffle(population)

        # Evaluate fitness for each genome in the population
        population_fitness = [(genome, fitness(genome, s, num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)) for genome in population]

        # Sort the population based on fitness
        sorted_population_fitness = sorted(population_fitness, key=lambda e: e[1], reverse=True)

        # Select top genomes for next generation
        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):

            # Function to look up fitness of a genome
            def fitness_lookup(genome):
                for e in population_fitness:
                    if e[0] == genome:
                        return e[1]
                return 0

            # Selection, crossover, and mutation
            parents = selection_pair(population, fitness_lookup)
            offspring_a, offspring_b = single_point_crossover(parents[0], parents[1])
            offspring_a = mutation(offspring_a, num=num_mutations, probability=mutation_probability)
            offspring_b = mutation(offspring_b, num=num_mutations, probability=mutation_probability)
            next_generation += [offspring_a, offspring_b]

        print(f"population {population_id} done")

        # Playback top two genomes
        events = genome_to_events(population[0], num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
        for e in events:
            e.play()
        s.start()
        input("here is the no1 hit …")
        s.stop()
        for e in events:
            e.stop()

        time.sleep(1)

        events = genome_to_events(population[1], num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
        for e in events:
            e.play()
        s.start()
        input("here is the second best …")
        s.stop()
        for e in events:
            e.stop()

        time.sleep(1)

        # Save MIDI files for each genome in the population
        print("saving population midi …")
        for i, genome in enumerate(population):
            save_genome_to_midi(f"{folder}/{population_id}/{scale}-{key}-{i}.mid", genome, num_bars, num_notes, num_steps, pauses, key, scale, root, bpm)
        print("done")

        # Prompt user to continue or stop
        running = input("continue? [Y/n]") != "n"
        population = next_generation
        population_id += 1


if __name__ == '__main__':
    main()

Number of bars [8]: 4
Notes per bar [4]: 2
Number of steps [1]: 1
Introduce Pauses (True/False) [False]: False
Key [C]: C
Scale [major]: major
Scale Root [4]: 4
Population size [10]: 5
Number of mutations [2]: 2
Mutation probability [0.5]: 0.5
BPM [128]: 128
Rating (0-5)5
Rating (0-5)2
Rating (0-5)3
Rating (0-5)1
Rating (0-5)4
here is the no1 hit …
here is the second best …
continue? [Y/n]y
